In [ ]:
!pip3 install pandas

In [2]:
!wget https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-08-Biomarker_disease.txt -O P1-08-Biomarker_disease.txt

--2024-04-16 12:10:41--  https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-08-Biomarker_disease.txt
Resolving db.idrblab.net (db.idrblab.net)... 47.88.56.212
Connecting to db.idrblab.net (db.idrblab.net)|47.88.56.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228662 (223K) [text/plain]
Saving to: ‘P1-08-Biomarker_disease.txt’

P1-08-Biomarker_dis 100%[===================>] 223.30K   752KB/s    in 0.3s    

2024-04-16 12:10:42 (752 KB/s) - ‘P1-08-Biomarker_disease.txt’ saved [228662/228662]



In [8]:
import pandas as pd

input_file_path = 'P1-08-Biomarker_disease.txt'
output_file_path = 'P1-08-Biomarker_disease_extracted.tsv'

try:
    comment_rows = range(0, 15)
    data = pd.read_csv(input_file_path, sep="\t", skiprows=comment_rows)

    data['ICD11'] = data['ICD11'].str.replace(': ', ':')

    new_data = pd.DataFrame({
        'source_id': data['BiomarkerID'],
        'source_type': 'Gene',
        'target_id': data['ICD11'],
        'target_type': 'Disease',
        'source_name': data['Biomarker_Name'],
        'target_name': data['Diseasename'],
        'relation_type': 'GNBR::Md::Gene:Disease',
        'resource': 'TTD',
        'ttd_disease_id': data['ICD11'],
        'ttd_biomarker_id': data['BiomarkerID']
    })

    new_data.to_csv(output_file_path, index=False, sep='\t')

    print('CSV file has been generated successfully!')
except Exception as e:
    print("Something went wrong: ", e)

CSV file has been generated successfully!


In [13]:
biomarkers = pd.read_csv("../idmapping/ttd_biomarker_id.tsv", sep='\t')
biomarkers.rename(columns={"UniProt ID": "uniprot_id", "Biomarker ID": "biomarker_id"}, inplace=True)
filtered_biomarkers = biomarkers[biomarkers['uniprot_id'].notnull()]

data = pd.read_csv(output_file_path, sep='\t')
data = data.merge(
    filtered_biomarkers[['biomarker_id', 'uniprot_id']],
    left_on="source_id", right_on="biomarker_id", how="inner"
)

data["source_id"] = "UNIPROT:" + data["uniprot_id"]
data = data.drop(columns=["biomarker_id", "uniprot_id"])

data.to_csv("formatted_ttd_biomarker_disease.tsv", index=False, sep='\t')